In [23]:
import torch
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import scipy.io
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch import Tensor
import matplotlib.pyplot as plt
from torch.nn import BatchNorm2d
from torch.nn import Dropout2d
from torch.nn import Sequential
from torch.nn import Linear
from torch.nn import Conv2d
from torch.nn import MaxPool2d
from torch.nn import Linear
from torch.nn import ReLU
from torch.nn import Softmax
from torch.nn import Module
from torch.optim import SGD, Adam
from torch.nn import CrossEntropyLoss, BCEWithLogitsLoss
from torch.nn.init import kaiming_uniform_
from torch.nn.init import xavier_uniform_
from torchvision.transforms import Compose
from torchvision.transforms import ToTensor
from torchvision.transforms import Normalize
from torchinfo import summary

In [7]:
train_transform = Compose(
    [ToTensor(),
    Normalize(mean=(0.1307,), std=(0.3081,))
    ])
test_transform = Compose(
    [ToTensor(),
    Normalize(mean=(0.1307,), std=(0.3081,))
    ])

In [8]:
BATCH_SIZE = 16

In [10]:
def get_data_from_mat(train_file,test_file):
    train_mat = scipy.io.loadmat(train_file) 
    test_mat = scipy.io.loadmat(test_file) 
    print(train_mat.keys())
    print(test_mat.keys())
    train_np = np.array(train_mat['train_data']).transpose(2,0,1)
    test_np = np.array(test_mat['test_data']).transpose(2,0,1)
    print('Train shape: ',train_np.shape)
    print('Test shape: ',test_np.shape)
    return  train_np, test_np


train_data, test_data = get_data_from_mat('train_data.mat','test_data.mat')

dict_keys(['__header__', '__version__', '__globals__', 'train_data'])
dict_keys(['__header__', '__version__', '__globals__', 'test_data'])
Train shape:  (112, 90, 90)
Test shape:  (28, 90, 90)


In [11]:
#Tornar cada matriz num vetor de uma dimensão (passar de 90x90 para 8100) e armazenar todos os valores de cada matriz
#O dicionário utilizado armazena, todos os valores com indíce X, pertencentes às diferentes 112 matrizes, sendo esse índice
#a chave do dic.
#Por exemplo: dictAttributes[0] contém o primeiro elemento de cada vetor anteriormente criado
dictAttributes = dict()
for i in range(len(train_data)):# len(train_data) - 112
    flatten_data = train_data[i].flatten()
    for j in range(len(flatten_data)):
        dictAttributes.setdefault(j,list())
        dictAttributes[j].append(flatten_data[j])
len(dictAttributes)

8100

In [12]:
#Nesta fase criam-se colunas com todos os elementos que estão numa dada posição do vetor, a label da coluna é essa posição
scans_series = []
for k in dictAttributes.keys():
    scans_series.append(pd.Series(dictAttributes.get(k)))

In [13]:
train_data_series = pd.concat(scans_series,axis=1)
train_data_series

,0,1,2,3,4,5,6,7,8,9,...,8090,8091,8092,8093,8094,8095,8096,8097,8098,8099
0,0.0,0.0,0.131525,0.0,0.0,0.0,0.271538,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.585006,0.0,0.280606,0.0,0.0
1,0.0,0.0,0.117704,0.0,0.0,0.0,0.316890,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.599437,0.0,0.207478,0.0,0.0
2,0.0,0.0,0.155806,0.0,0.0,0.0,0.348072,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.630374,0.0,0.246472,0.0,0.0
3,0.0,0.0,0.145214,0.0,0.0,0.0,0.310270,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.562505,0.0,0.287888,0.0,0.0
4,0.0,0.0,0.161360,0.0,0.0,0.0,0.385560,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.564043,0.0,0.234284,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,0.0,0.0,0.215767,0.0,0.0,0.0,0.390332,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.011150,0.0,0.660292,0.0,0.302675,0.0,0.0
108,0.0,0.0,0.143005,0.0,0.0,0.0,0.306380,0.0,0.0,0.0,...,0.0,0.011171,0.0,0.019141,0.0,0.630415,0.0,0.204125,0.0,0.0
109,0.0,0.0,0.116085,0.0,0.0,0.0,0.290941,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.574355,0.0,0.246784,0.0,0.0
110,0.0,0.0,0.156654,0.0,0.0,0.0,0.357540,0.0,0.0,0.0,...,0.0,0.010225,0.0,0.000000,0.0,0.631057,0.0,0.216259,0.0,0.0


In [14]:
TRAIN_CSV_PATH = 'train.csv'

In [15]:
rest_train_df = pd.read_csv(TRAIN_CSV_PATH)
rest_train_df.head(10) #Tem 112 linhas

,id,age,sex,education
0,1,13,1,7
1,2,14,0,8
2,3,15,1,9
3,4,15,1,9
4,5,15,1,9
5,6,15,1,9
6,7,16,0,10
7,8,16,1,10
8,9,16,0,10
9,10,17,0,11


In [16]:
targets_train = rest_train_df['age']
targets_train

0      13
1      14
2      15
3      15
4      15
       ..
107    77
108    67
109    55
110    76
111    69
Name: age, Length: 112, dtype: int64

In [17]:
train_data_series = pd.concat([targets_train] + scans_series,axis=1).sample(frac=1.0, random_state=2022).reset_index(drop=True)
train_data_series

,age,0,1,2,3,4,5,6,7,8,...,8090,8091,8092,8093,8094,8095,8096,8097,8098,8099
0,53,0.0,0.0,0.173673,0.019753,0.0,0.0,0.399909,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.615459,0.0,0.113210,0.0,0.0
1,40,0.0,0.0,0.122071,0.000000,0.0,0.0,0.295469,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.596293,0.0,0.256004,0.0,0.0
2,69,0.0,0.0,0.330159,0.000000,0.0,0.0,0.389147,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.676865,0.0,0.197393,0.0,0.0
3,24,0.0,0.0,0.144803,0.000000,0.0,0.0,0.352847,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.653635,0.0,0.227650,0.0,0.0
4,55,0.0,0.0,0.116085,0.000000,0.0,0.0,0.290941,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.574355,0.0,0.246784,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,63,0.0,0.0,0.153905,0.000000,0.0,0.0,0.272740,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.577098,0.0,0.194381,0.0,0.0
108,61,0.0,0.0,0.178360,0.000000,0.0,0.0,0.306510,0.0,0.0,...,0.0,0.011916,0.0,0.0,0.0,0.723154,0.0,0.200323,0.0,0.0
109,22,0.0,0.0,0.134661,0.000000,0.0,0.0,0.361852,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.532999,0.0,0.238360,0.0,0.0
110,27,0.0,0.0,0.116392,0.000000,0.0,0.0,0.303962,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.570106,0.0,0.231983,0.0,0.0


In [18]:
train_df_all = pd.DataFrame(train_data_series)
train_df_all

,age,0,1,2,3,4,5,6,7,8,...,8090,8091,8092,8093,8094,8095,8096,8097,8098,8099
0,53,0.0,0.0,0.173673,0.019753,0.0,0.0,0.399909,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.615459,0.0,0.113210,0.0,0.0
1,40,0.0,0.0,0.122071,0.000000,0.0,0.0,0.295469,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.596293,0.0,0.256004,0.0,0.0
2,69,0.0,0.0,0.330159,0.000000,0.0,0.0,0.389147,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.676865,0.0,0.197393,0.0,0.0
3,24,0.0,0.0,0.144803,0.000000,0.0,0.0,0.352847,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.653635,0.0,0.227650,0.0,0.0
4,55,0.0,0.0,0.116085,0.000000,0.0,0.0,0.290941,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.574355,0.0,0.246784,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,63,0.0,0.0,0.153905,0.000000,0.0,0.0,0.272740,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.577098,0.0,0.194381,0.0,0.0
108,61,0.0,0.0,0.178360,0.000000,0.0,0.0,0.306510,0.0,0.0,...,0.0,0.011916,0.0,0.0,0.0,0.723154,0.0,0.200323,0.0,0.0
109,22,0.0,0.0,0.134661,0.000000,0.0,0.0,0.361852,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.532999,0.0,0.238360,0.0,0.0
110,27,0.0,0.0,0.116392,0.000000,0.0,0.0,0.303962,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.570106,0.0,0.231983,0.0,0.0


In [19]:
class MyDataset(Dataset):

    def __init__(self, df, transform=None):
        self.transform = transform

        df_set = df

        self.x = df_set.values[:, 1:]
        self.y = df_set.values[:, 0]

        self.x = self.x.astype('float32')
        self.y = self.y.astype('long')
        
    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        label = self.y[idx]
        image = self.x[idx]       
        if self.transform is not None:
            image = self.transform(image)
        return image, label
    
    def get_TensorDataset(self):
        x = self.x.reshape(len(self.x), 1, 90, 90)
        xmax, xmin = x.max(), x.min()
        x = (x - xmin)/(xmax - xmin)
        x  = torch.from_numpy(np.array(x)).float()
        y  = torch.from_numpy(np.array(self.y)).type(torch.LongTensor)
        cases = torch.utils.data.TensorDataset(x,y)
        return cases 

def prepare_data_loaders(df_train):
    dataset_train = MyDataset(df_train,transform=train_transform)
    #dataset_test = MyDataset(path_test,transform=test_transform)
    train = dataset_train.get_TensorDataset()
    train_size = int(0.8 * len(train))
    val_size = len(train) - train_size
    train, validation = random_split(train, [train_size, val_size], generator=torch.Generator().manual_seed(42))
    #test = dataset_test.get_TensorDataset()
    
    train_dl = DataLoader(train, batch_size=BATCH_SIZE, shuffle=True)
    val_dl = DataLoader(validation, batch_size=BATCH_SIZE, shuffle=True)
    #test_dl = DataLoader(test, batch_size=BATCH_SIZE, shuffle=True)
    train_dl_all = DataLoader(train, batch_size=len(train), shuffle=False)
    val_dl_all = DataLoader(validation, batch_size=len(validation), shuffle=True)
    #test_dl_all = DataLoader(test, batch_size=len(test), shuffle=False)
    return train_dl, val_dl, train_dl_all, val_dl_all

train_dl, val_dl, train_dl_all, val_dl_all = prepare_data_loaders(train_df_all)

In [25]:
class CNNModel_1(Module):
    def __init__(self):
        super(CNNModel_1, self).__init__()
        self.layer1 = Sequential(Conv2d(in_channels=1, out_channels=32, kernel_size=(3,3)),
                                 ReLU(),
                                 MaxPool2d(kernel_size=(2,2), stride=(2,2))           
                                )
        self.layer2 = Sequential(Conv2d(in_channels=32, out_channels=32, kernel_size=(3,3)),
                                 ReLU(),
                                 MaxPool2d(kernel_size=(2,2), stride=(2,2))           
                                )
        self.fc1 = Linear(in_features=21*21*32, out_features=100)
        kaiming_uniform_(self.fc1.weight, nonlinearity='relu')
        self.act1 = ReLU()
        self.fc2 = Linear(in_features=100, out_features=10)
        xavier_uniform_(self.fc2.weight)
        self.act2 = Softmax(dim=1)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(-1, 4*4*50)
        out = self.fc1(out)
        out = self.act1(out)
        out = self.fc2(out) 
        out = self.act2(out)
        return out

model = CNNModel_1()
print(summary(model, input_size=(BATCH_SIZE, 1,90,90), verbose=0))
model.to(device)

RuntimeError: Failed to run torchinfo. See above stack traces for more details. Executed layers up to: [Sequential: 1, Conv2d: 2, ReLU: 2, MaxPool2d: 2, Sequential: 1, Conv2d: 2, ReLU: 2, MaxPool2d: 2]